In [1]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr='127.0.0.1:8001',namespace='mdt')
clipper_conn = ClipperConnection(manager)
clipper_conn.start_clipper()

18-10-01:09:54:41 INFO     [kubernetes_container_manager.py:317] [default-cluster] Found 6 nodes: ec2-54-68-64-106.us-west-2.compute.amazonaws.com, ec2-34-219-191-153.us-west-2.compute.amazonaws.com, ec2-18-236-197-218.us-west-2.compute.amazonaws.com, ec2-54-212-98-152.us-west-2.compute.amazonaws.com, ec2-18-236-106-32.us-west-2.compute.amazonaws.com, ec2-52-13-37-198.us-west-2.compute.amazonaws.com
18-10-01:09:54:41 INFO     [kubernetes_container_manager.py:328] [default-cluster] Setting Clipper mgmt port to 31748
18-10-01:09:54:41 INFO     [kubernetes_container_manager.py:338] [default-cluster] Setting Clipper query port to 31925
18-10-01:09:54:41 INFO     [kubernetes_container_manager.py:361] [default-cluster] Setting Clipper metric port to 30271
18-10-01:09:54:41 INFO     [clipper_admin.py:136] [default-cluster] Clipper still initializing.
18-10-01:09:54:42 INFO     [clipper_admin.py:136] [default-cluster] Clipper still initializing.
18-10-01:09:54:43 INFO     [clipper_admin.py:136

In [5]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr='127.0.0.1:8001',namespace='mdt')
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()
#clipper_conn.register_application(name="hello-world", input_type="doubles", default_output="-1.0", slo_micros=100000)

18-10-01:09:57:44 INFO     [kubernetes_container_manager.py:317] [default-cluster] Found 6 nodes: ec2-54-68-64-106.us-west-2.compute.amazonaws.com, ec2-34-219-191-153.us-west-2.compute.amazonaws.com, ec2-18-236-197-218.us-west-2.compute.amazonaws.com, ec2-54-212-98-152.us-west-2.compute.amazonaws.com, ec2-18-236-106-32.us-west-2.compute.amazonaws.com, ec2-52-13-37-198.us-west-2.compute.amazonaws.com
18-10-01:09:57:44 INFO     [kubernetes_container_manager.py:328] [default-cluster] Setting Clipper mgmt port to 31748
18-10-01:09:57:44 INFO     [kubernetes_container_manager.py:338] [default-cluster] Setting Clipper query port to 31925
18-10-01:09:57:44 INFO     [kubernetes_container_manager.py:361] [default-cluster] Setting Clipper metric port to 30271
18-10-01:09:57:44 INFO     [clipper_admin.py:151] [default-cluster] Successfully connected to Clipper cluster at 127.0.0.1:8001/api/v1/namespaces/mdt/services/query-frontend-at-default-cluster:1337/proxy


In [14]:
clipper_conn.register_application(name="hello-world", input_type="doubles", default_output="-1.0", slo_micros=100000)
print clipper_conn.get_all_apps()

18-10-01:10:10:51 INFO     [clipper_admin.py:215] [default-cluster] Application hello-world was successfully registered


[u'hello-world']


In [12]:
!$(aws ecr get-login --no-include-email --region us-west-2)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


In [16]:
from clipper_admin.deployers import python as python_deployer
from clipper_admin import ClipperConnection, KubernetesContainerManager

manager = KubernetesContainerManager(kubernetes_proxy_addr='127.0.0.1:8001',namespace='mdt')
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()
def feature_sum(xs):
    return [str(sum(x)) for x in xs]

python_deployer.deploy_python_closure(clipper_conn, 
                                      name="sum-model", 
                                      version=2, 
                                      input_type="doubles", 
                                      func=feature_sum,
                                     registry='658391232643.dkr.ecr.us-west-2.amazonaws.com')

18-10-01:10:11:39 INFO     [kubernetes_container_manager.py:317] [default-cluster] Found 6 nodes: ec2-54-68-64-106.us-west-2.compute.amazonaws.com, ec2-34-219-191-153.us-west-2.compute.amazonaws.com, ec2-18-236-197-218.us-west-2.compute.amazonaws.com, ec2-54-212-98-152.us-west-2.compute.amazonaws.com, ec2-18-236-106-32.us-west-2.compute.amazonaws.com, ec2-52-13-37-198.us-west-2.compute.amazonaws.com
18-10-01:10:11:39 INFO     [kubernetes_container_manager.py:328] [default-cluster] Setting Clipper mgmt port to 31748
18-10-01:10:11:39 INFO     [kubernetes_container_manager.py:338] [default-cluster] Setting Clipper query port to 31925
18-10-01:10:11:39 INFO     [kubernetes_container_manager.py:361] [default-cluster] Setting Clipper metric port to 30271
18-10-01:10:11:39 INFO     [clipper_admin.py:151] [default-cluster] Successfully connected to Clipper cluster at 127.0.0.1:8001/api/v1/namespaces/mdt/services/query-frontend-at-default-cluster:1337/proxy
18-10-01:10:11:39 INFO     [deployer

In [17]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="sum-model")

18-10-01:10:13:43 INFO     [clipper_admin.py:277] [default-cluster] Model sum-model is now linked to application hello-world


In [19]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:8001/api/v1/namespaces/mdt/services/query-frontend-at-default-cluster:1337/proxy/hello-world/predict", headers=headers, data=json.dumps({"input": list(np.random.random(10))})).json()


{u'default': True,
 u'default_explanation': u'No connected models found for query',
 u'output': -1.0,
 u'query_id': 0}

In [ ]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
manager = KubernetesContainerManager(kubernetes_proxy_addr='127.0.0.1:8001',namespace='mdt')
clipper_conn = ClipperConnection(manager)
clipper_conn.stop_all()